# Fundamentos de Aprendizaje Automático


## Práctica 2: kNN & Regresión Logística


In [7]:
# Imports globales
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

import compileall
import numpy as np
import time
import matplotlib.pyplot as plt

compileall.compile_dir('.',force=True,quiet=True)

from datos import Datos
from estrategiasparticionado.validacion_cruzada import ValidacionCruzada
from estrategiasparticionado.validacion_simple import ValidacionSimple
from plotModel import plotModel

%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Apartado 1 
### Resultados   de   la   clasificación   mediante   vecinos   próximos (implementación  original)  para  diferentes  valores  de  vecindad  en los   conjuntos   de   datos   propuestos.   Comparación   con los resultados proporcionados por scikit-learn. Obtener los resultados tanto para datos normalizados como sin normalizar. 

En primer lugar vamos a comparar el rendimiento de nuestro clasificador KNN con el clasificador implementado en scikit-learn.

Comentar que nuestra implementación es considerablemente más lenta, dado que no se paraleliza el cálculo de distancias, que resulta ser la tarea más costosaen clasificación

Vamos a hacer este experimento con ambos clasificadores para ordenes de vecindad K = 1, 3, 5, 11, 21 y 51, y con los conjuntos de datos example1.data, example2.data, example3.data y example4.data.

El método de validación en todos los casos será validación simple.

In [ ]:
Ks = [1,3,4,11,21,51]
    
for K in Ks:
    for i in range(1, 5):
        # Inicialización
        nombre_fichero = './conjunto_datos/example%d.data' % i
        dataset=Datos(nombre_fichero)
        estrategia = ValidacionSimple()
        estrategia.creaParticiones(dataset.datos)
        
        clasificador = ClasificadorVecinosProximos(K=K)
        
        # Validación implementación propia
        res = clasificador.validacion(estrategia, dataset)
        

In [9]:
from clasificadores.clasificador_vecinos_proximos import ClasificadorVecinosProximos

start = time.clock()

clasificador = ClasificadorVecinosProximos()

for i in range(1, 5):
    nombre_fichero = './conjunto_datos/example%d.data' % i
    
    dataset=Datos(nombre_fichero)
    estrategia = ValidacionSimple()
    estrategia.creaParticiones(dataset.datos)
    res = clasificador.validacion(estrategia, dataset)
    ii = estrategia.particiones[-1].indicesTrain
    plt.figure()
    
    plotModel(dataset.datos[ii,0], dataset.datos[ii,1], dataset.datos[ii,-1]!=0,
              clasificador, nombre_fichero, dataset.diccionarios)
    
    plt.show()
    print('Error de ./conjunto_datos/example%d.data: %s'% (i, clasificador.errores[0]))
    
print("Tiempo total: %2.3f" % (time.clock()-start))

KeyboardInterrupt: 

#### Implementación de scikit

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn import preprocessing 
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import ShuffleSplit
from matplotlib.colors import ListedColormap

start = time.clock()
for i in range(1, 5):
    nombre_fichero = './conjunto_datos/example%d.data' % i
    dataset=Datos(nombre_fichero)
    estrategia = ValidacionSimple()
    X = dataset.datos[:,:-1]
    Y = dataset.datos[:,-1]

    Y = dataset.datos[:,-1] 
    class_names = sorted(dataset.diccionarios[-1].keys())
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25, random_state=29)


    model = KNeighborsClassifier(n_neighbors=5,p=2,algorithm='brute')
    model.fit(X_train, Y_train)

    h = .02
    x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
    y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
    
    # Put the result into a color plot
    Z = Z.reshape(xx.shape)
    plt.figure(1, figsize=(4, 3))
    cm = plt.cm.RdBu
    cm_bright = ListedColormap(['#FF0000', '#0000FF'])
    plt.pcolormesh(xx, yy, Z, cmap=cm, alpha=0.8)

    # Plot also the training points
    plt.scatter(X[:, 0], X[:, 1], c=Y, edgecolors='k', cmap=cm_bright)
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.title(nombre_fichero)
    
    plt.xlim(xx.min(), xx.max())
    plt.ylim(yy.min(), yy.max())
    plt.xticks(())
    plt.yticks(())

    plt.show()

    model.fit(X_train, Y_train)
    score = model.score(X_test, Y_test)
    print("Validación simple. Error: %0.2f" % (1-score))
    print("Tiempo:",time.clock() - step)
    
print("Tiempo total: %2.3f" % (time.clock()-start))

### Apartado b) Regresión logística
#### Implementación propia

In [ ]:
from clasificadores.clasificador_regresion_logistica import ClasificadorRegresionLogistica

start = time.clock()
clasificador = ClasificadorRegresionLogistica()

for i in range(1, 5):
    step = time.clock()
    nombre_fichero = './conjunto_datos/example%d.data' % i
    dataset=Datos(nombre_fichero)
    estrategia = ValidacionSimple()
    res = clasificador.validacion(estrategia, dataset)
    estrategia.creaParticiones(dataset.datos)
    ii = estrategia.particiones[-1].indicesTrain
    plt.figure()
    plotModel(dataset.datos[ii,0], dataset.datos[ii,1], dataset.datos[ii,-1]!=0,
              clasificador, nombre_fichero, dataset.diccionarios)
    plt.show()
    print('Error de ./conjunto_datos/example%d.data: %s'% (i, clasificador.errores[0]))
    print("Tiempo:",time.clock() - step)
    
print("Tiempo total: %2.3f" % (time.clock()-start))



#### Implementación de scikit

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import preprocessing 
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import ShuffleSplit
from matplotlib.colors import ListedColormap

start = time.clock()

for i in range(1, 5):
    step = time.clock()
    nombre_fichero = './conjunto_datos/example%d.data' % i
    dataset=Datos(nombre_fichero)
    estrategia = ValidacionSimple()
    X = dataset.datos[:,:-1]
    Y = dataset.datos[:,-1]

    Y = dataset.datos[:,-1] 
    class_names = sorted(dataset.diccionarios[-1].keys())
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25, random_state=29)


    model = LogisticRegression(C=1e5)
    model.fit(X_train, Y_train)

    h = .02
    x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
    y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
    # Put the result into a color plot
    Z = Z.reshape(xx.shape)
    plt.figure(1, figsize=(4, 3))
    cm = plt.cm.RdBu
    cm_bright = ListedColormap(['#FF0000', '#0000FF'])
    plt.pcolormesh(xx, yy, Z, cmap=cm, alpha=0.8)

    # Plot also the training points
    plt.scatter(X[:, 0], X[:, 1], c=Y, edgecolors='k', cmap=cm_bright)
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.title(nombre_fichero)
    
    plt.xlim(xx.min(), xx.max())
    plt.ylim(yy.min(), yy.max())
    plt.xticks(())
    plt.yticks(())

    plt.show()

    model.fit(X_train, Y_train)
    score = model.score(X_test, Y_test)
    print("Validación simple. Error: %0.2f" % (1-score))
    print("Tiempo:",time.clock() - step)
    
print("Tiempo total: %2.3f" % (time.clock()-start))

#### Comentario sobre los resultados
Se observa como para los dataset 1, 2 y 3, como no se puede separar los conjuntos por una frontera lineal, el error es bastante alto y la frontera lineal es la máxima posible dada la limitación de la frontera lineal.